In [1]:
import findspark
findspark.find()
import pyspark

In [2]:
findspark.init('/usr/local/opt/apache-spark/libexec')

In [3]:
sc = pyspark.SparkContext()

In [4]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

In [5]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(values[0], values[1:])


In [6]:
data = sc.textFile("lpsa.data")
parsedData = data.map(parsePoint)

In [10]:
parsedData.count()

67

In [14]:
parsedData.take(4)

[LabeledPoint(-0.4307829, [-1.63735562648104,-2.00621178480549,-1.86242597251066,-1.02470580167082,-0.522940888712441,-0.863171185425945,-1.04215728919298,-0.864466507337306]),
 LabeledPoint(-0.1625189, [-1.98898046126935,-0.722008756122123,-0.787896192088153,-1.02470580167082,-0.522940888712441,-0.863171185425945,-1.04215728919298,-0.864466507337306]),
 LabeledPoint(-0.1625189, [-1.57881887548545,-2.1887840293994,1.36116336875686,-1.02470580167082,-0.522940888712441,-0.863171185425945,0.342627053981254,-0.155348103855541]),
 LabeledPoint(-0.1625189, [-2.16691708463163,-0.807993896938655,-0.787896192088153,-1.02470580167082,-0.522940888712441,-0.863171185425945,-1.04215728919298,-0.864466507337306])]

In [7]:
# Build the model
model = LinearRegressionWithSGD.train(parsedData, iterations=100, step=0.00000001)

/Users/xiedong/anaconda/lib/python3.6/site-packages/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [8]:
# Evaluate the model on training data
valuesAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 7.4510328101026015


In [9]:
# Save and load model
model.save(sc, "pythonLinearRegressionWithSGDModel")
sameModel = LinearRegressionModel.load(sc, "pythonLinearRegressionWithSGDModel")